# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298384598751                   -0.85    5.0
  2   -8.300180483141       -2.75       -1.25    1.2
  3   -8.300444536526       -3.58       -1.89    3.1
  4   -8.300462087386       -4.76       -2.76    1.5
  5   -8.300464267625       -5.66       -3.08    5.6
  6   -8.300464441092       -6.76       -3.25    1.0
  7   -8.300464537341       -7.02       -3.39    2.4
  8   -8.300464588168       -7.29       -3.51    1.0
  9   -8.300464626224       -7.42       -3.69    1.0
 10   -8.300464639021       -7.89       -3.87    1.1
 11   -8.300464643706       -8.33       -4.29    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67441386091                   -0.70    6.6
  2   -16.67865307905       -2.37       -1.14    1.5
  3   -16.67922646226       -3.24       -1.87    3.4
  4   -16.67928114040       -4.26       -2.77    3.2
  5   -16.67928611070       -5.30       -3.21    5.2
  6   -16.67928621872       -6.97       -3.52    3.5
  7   -16.67928622299       -8.37       -3.99    1.0
  8   -16.67928622429       -8.89       -4.62    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32667363531                   -0.56    7.4
  2   -33.33435063657       -2.11       -1.00    1.5
  3   -33.33603588831       -2.77       -1.72    7.6
  4   -33.33617248136       -3.86       -2.63    4.4
  5   -33.33687807385       -3.15       -2.41    5.5
  6   -33.33690574159       -4.56       -2.54    3.8
  7   -33.33694376784       -4.42       -3.68    2.5
  8   -33.33694387734       -6.96       -3.88    5.6
  9   -33.33694389404       -7.78       -4.08    2.1


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298623583414                   -0.85    4.9
  2   -8.300264695036       -2.78       -1.58    1.2
  3   -8.300284799274       -4.70       -2.11    5.2
  4   -8.300346926270       -4.21       -2.22    4.6
  5   -8.300464077210       -3.93       -3.49    1.0
  6   -8.300464583608       -6.30       -3.92    3.5
  7   -8.300464640558       -7.24       -4.46    2.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32656126644                   -0.56    7.4
  2   -33.31049648778   +   -1.79       -1.27    1.4
  3   -10.05493198382   +    1.37       -0.38    8.2
  4   -33.33105571841        1.37       -1.82    6.4
  5   -33.22522115071   +   -0.98       -1.30    4.5
  6   -33.08008898427   +   -0.84       -1.36    4.6
  7   -33.30249311549       -0.65       -1.78    4.8
  8   -33.33589827051       -1.48       -2.47    3.6
  9   -33.33620816629       -3.51       -2.32    2.8
 10   -33.33683040598       -3.21       -2.91    2.2
 11   -33.33690887526       -4.11       -3.16    2.4
 12   -33.33692453009       -4.81       -3.37    2.0
 13   -33.33694377035       -4.72       -4.07    3.2


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.